In [1]:
pip install ollama

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from api import chat_with_model, SUPPORTED_MODELS
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

In [2]:
# === CONFIG ===
PROMPT_DIR = "./prompt_templates"
PROMPT_FILES = ["PS1.txt", "PS2.txt", "PS3.txt", "PS4.txt", "PS5.txt"]

In [3]:
TOPICS = [
    "Decision Tree Models",
    "Training, Validation, and Testing of Machine Learning Models",
    "Gradient Boosted Tree Models",
    "Linear Regression, Logistic Regression, and Multilayer Perceptron",
    "Stochastic Gradient Descent",
    "Backpropagation",
    "Foundations of Computer Vision and Convolutional Neural Networks",
    "Transfer Learning for Computer Vision",
    "Image Segmentation and Object Detection",
    "Data Pre-processing for Natural Language Processing Tasks",
    "Bag of Words Approach and Word Embedding",
    "Attention Mechanism in Transformers",
    "Neural Machine Translation using Transformers",
    "Encoder, Decoder, and Sequence-to-Sequence Transformers",
    "Pretraining, Finetuning, and Reinforcement Learning with Human Feedback",
    "Prompt Engineering and Chain of Thought Prompting",
    "Natural Language Processing Tasks and Transformer Architectures Used for the Tasks"
]

In [4]:
default_options = {
    "temperature": 0.8,
    "top_p": 0.9,
    "top_k": 40,
    "num_predict": -1,
    "presence_penalty": 0.0,
    "frequency_penalty": 0.0,
    "repeat_penalty": 1.1,
    "stop": [],
    "seed": None,
    "mirostat": 0,
    "mirostat_tau": 5.0,
    "mirostat_eta": 0.1,
    "grammar": None
}

In [5]:
def fill_prompt_template(template_text, values_dict):
    for key, value in values_dict.items():
        template_text = template_text.replace(f"{{{key}}}", value)
    return template_text

def generate_questions_for_topic(topic):
    return {
        "topic": topic,
        "Remember_question": f"What is {topic}?",
        "Understand_question": f"Can you explain the key ideas behind {topic}?",
        "Apply_question": f"How would you apply {topic} in a real-world scenario?",
        "Analyze_question": f"Analyze the components and working of {topic}.",
        "Evaluate_question": f"Evaluate the strengths and limitations of {topic}.",
        "Create_question": f"Design an experiment or application using {topic}."
    }

In [6]:
wb = Workbook()
ws = wb.active
ws.title = "Model Responses"

# Set headers
ws.append(["Index", "Prompt", "Model", "Responses"])

In [ ]:
row_index = 2
index = 1

for prompt_file in PROMPT_FILES:
    with open(os.path.join(PROMPT_DIR, prompt_file), "r", encoding="utf-8") as f:
        template_text = f.read()

    for topic in TOPICS:
        input_data = generate_questions_for_topic(topic)
        filled_prompt = fill_prompt_template(template_text, input_data)

        # First row for this topic gets the prompt text
        first_entry = True
        start_row = row_index

        for model in SUPPORTED_MODELS:
            print(f"🧠 Running: {prompt_file} | Topic: {topic} | Model: {model}")
            messages = [
                {"role": "system", "content": "You are a helpful assistant."}, # For Future Experiements include output formatting (eg. JSON) for easy parsing
                {"role": "user", "content": filled_prompt}
            ]

            try:
                response = chat_with_model(
                    model_name=model,
                    messages=messages,
                    options=default_options
                )
                answer = response["message"]["content"].strip()
            except Exception as e:
                answer = f"Error: {e}"

            ws.append([index, filled_prompt if first_entry else "", model, answer])
            index += 1
            row_index += 1
            first_entry = False

🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: granite3.2:2b
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: phi4-mini:latest
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: llama3.2:latest
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: mistral-small:22b
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: phi4:latest
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: deepseek-r1:14b
🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: gemma3:latest
🧠 Running: PS1.txt | Topic: Training, Validation, and Testing of Machine Learning Models | Model: granite3.2:2b
🧠 Running: PS1.txt | Topic: Training, Validation, and Testing of Machine Learning Models | Model: phi4-mini:latest
🧠 Running: PS1.txt | Topic: Training, Validation, and Testing of Machine Learning Models | Model: llama3.2:latest
🧠 Running: PS1.txt | Topic: Training, Validation, and Testing of Machine Learning Models | Model: mistral-small:22b
🧠 Running: PS1.txt | Topic: Tr

In [10]:
# Optional: auto-size columns
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max(15, min(max_length + 2, 50))

In [11]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def merge_column_b_rows(workbook, n):
    sheet = workbook.active
    max_row = sheet.max_row

    start_row = 2
    while start_row <= max_row:
        end_row = min(start_row + n - 1, max_row)
        if start_row != end_row:
            cell_range = f"B{start_row}:B{end_row}"
            sheet.merge_cells(cell_range)
        start_row += n

In [12]:
merge_column_b_rows(wb, len(SUPPORTED_MODELS))

In [13]:
# Save the workbook
output_file = "Model_Responses.xlsx"
wb.save(output_file)
print(f"\n✅ Excel spreadsheet saved as {output_file}")


✅ Excel spreadsheet saved as Model_Responses.xlsx


In [10]:
import csv
for prompt_file in PROMPT_FILES:
    with open(os.path.join(PROMPT_DIR, prompt_file), "r", encoding="utf-8") as f:
        template_text = f.read()
    temp_list = []
    for topic in TOPICS:
        input_data = generate_questions_for_topic(topic)
        filled_prompt = fill_prompt_template(template_text, input_data)
        temp_list.append(filled_prompt)
    print(temp_list)
    print("\n\n\n")

['Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Decision Tree Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Training, Validation, and Testing of Machine Learning Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Gradient Boosted Tree Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Linear Regression, Logistic Regression, and Multilayer Perceptron.\n\nPlease make sure that the questi

In [13]:
from openpyxl import load_workbook

# Path to your Excel file
file_path = 'PS1_QualityQuestions (1).xlsx'

# Data to fill for Column B in each sheet
data_to_fill = {
    'PS1': ['Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Decision Tree Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Training, Validation, and Testing of Machine Learning Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Gradient Boosted Tree Models.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Linear Regression, Logistic Regression, and Multilayer Perceptron.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Stochastic Gradient Descent.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Backpropagation.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Foundations of Computer Vision and Convolutional Neural Networks.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Transfer Learning for Computer Vision.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Image Segmentation and Object Detection.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Data Pre-processing for Natural Language Processing Tasks.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Bag of Words Approach and Word Embedding.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Attention Mechanism in Transformers.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Neural Machine Translation using Transformers.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Encoder, Decoder, and Sequence-to-Sequence Transformers.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Prompt Engineering and Chain of Thought Prompting.\n\nPlease make sure that the questions are relatable to students in India.', 'Create a question of each cognitive level in revised Bloom’s taxonomy: remember, understand, apply, analyze, evaluate, and create on the Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n\nPlease make sure that the questions are relatable to students in India.'],
    'PS2': ['You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Decision Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Training, Validation, and Testing of Machine Learning Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Gradient Boosted Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Linear Regression, Logistic Regression, and Multilayer Perceptron. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Stochastic Gradient Descent. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Backpropagation. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Foundations of Computer Vision and Convolutional Neural Networks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Transfer Learning for Computer Vision. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Image Segmentation and Object Detection. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Data Pre-processing for Natural Language Processing Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Bag of Words Approach and Word Embedding. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Attention Mechanism in Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Neural Machine Translation using Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Encoder, Decoder, and Sequence-to-Sequence Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Pretraining, Finetuning, and Reinforcement Learning with Human Feedback. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Prompt Engineering and Chain of Thought Prompting. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels and their explanation:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Natural Language Processing Tasks and Transformer Architectures Used for the Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.'],
    'PS3': ['You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Decision Tree Models?\n2. Example: Can you explain the key ideas behind Decision Tree Models?\n3. Example: How would you apply Decision Tree Models in a real-world scenario?\n4. Example: Analyze the components and working of Decision Tree Models.\n5. Example: Evaluate the strengths and limitations of Decision Tree Models.\n6. Example: Design an experiment or application using Decision Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Decision Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Training, Validation, and Testing of Machine Learning Models?\n2. Example: Can you explain the key ideas behind Training, Validation, and Testing of Machine Learning Models?\n3. Example: How would you apply Training, Validation, and Testing of Machine Learning Models in a real-world scenario?\n4. Example: Analyze the components and working of Training, Validation, and Testing of Machine Learning Models.\n5. Example: Evaluate the strengths and limitations of Training, Validation, and Testing of Machine Learning Models.\n6. Example: Design an experiment or application using Training, Validation, and Testing of Machine Learning Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Training, Validation, and Testing of Machine Learning Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Gradient Boosted Tree Models?\n2. Example: Can you explain the key ideas behind Gradient Boosted Tree Models?\n3. Example: How would you apply Gradient Boosted Tree Models in a real-world scenario?\n4. Example: Analyze the components and working of Gradient Boosted Tree Models.\n5. Example: Evaluate the strengths and limitations of Gradient Boosted Tree Models.\n6. Example: Design an experiment or application using Gradient Boosted Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Gradient Boosted Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Linear Regression, Logistic Regression, and Multilayer Perceptron?\n2. Example: Can you explain the key ideas behind Linear Regression, Logistic Regression, and Multilayer Perceptron?\n3. Example: How would you apply Linear Regression, Logistic Regression, and Multilayer Perceptron in a real-world scenario?\n4. Example: Analyze the components and working of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n5. Example: Evaluate the strengths and limitations of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n6. Example: Design an experiment or application using Linear Regression, Logistic Regression, and Multilayer Perceptron.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Linear Regression, Logistic Regression, and Multilayer Perceptron. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Stochastic Gradient Descent?\n2. Example: Can you explain the key ideas behind Stochastic Gradient Descent?\n3. Example: How would you apply Stochastic Gradient Descent in a real-world scenario?\n4. Example: Analyze the components and working of Stochastic Gradient Descent.\n5. Example: Evaluate the strengths and limitations of Stochastic Gradient Descent.\n6. Example: Design an experiment or application using Stochastic Gradient Descent.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Stochastic Gradient Descent. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Backpropagation?\n2. Example: Can you explain the key ideas behind Backpropagation?\n3. Example: How would you apply Backpropagation in a real-world scenario?\n4. Example: Analyze the components and working of Backpropagation.\n5. Example: Evaluate the strengths and limitations of Backpropagation.\n6. Example: Design an experiment or application using Backpropagation.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Backpropagation. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Foundations of Computer Vision and Convolutional Neural Networks?\n2. Example: Can you explain the key ideas behind Foundations of Computer Vision and Convolutional Neural Networks?\n3. Example: How would you apply Foundations of Computer Vision and Convolutional Neural Networks in a real-world scenario?\n4. Example: Analyze the components and working of Foundations of Computer Vision and Convolutional Neural Networks.\n5. Example: Evaluate the strengths and limitations of Foundations of Computer Vision and Convolutional Neural Networks.\n6. Example: Design an experiment or application using Foundations of Computer Vision and Convolutional Neural Networks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Foundations of Computer Vision and Convolutional Neural Networks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Transfer Learning for Computer Vision?\n2. Example: Can you explain the key ideas behind Transfer Learning for Computer Vision?\n3. Example: How would you apply Transfer Learning for Computer Vision in a real-world scenario?\n4. Example: Analyze the components and working of Transfer Learning for Computer Vision.\n5. Example: Evaluate the strengths and limitations of Transfer Learning for Computer Vision.\n6. Example: Design an experiment or application using Transfer Learning for Computer Vision.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Transfer Learning for Computer Vision. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Image Segmentation and Object Detection?\n2. Example: Can you explain the key ideas behind Image Segmentation and Object Detection?\n3. Example: How would you apply Image Segmentation and Object Detection in a real-world scenario?\n4. Example: Analyze the components and working of Image Segmentation and Object Detection.\n5. Example: Evaluate the strengths and limitations of Image Segmentation and Object Detection.\n6. Example: Design an experiment or application using Image Segmentation and Object Detection.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Image Segmentation and Object Detection. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Data Pre-processing for Natural Language Processing Tasks?\n2. Example: Can you explain the key ideas behind Data Pre-processing for Natural Language Processing Tasks?\n3. Example: How would you apply Data Pre-processing for Natural Language Processing Tasks in a real-world scenario?\n4. Example: Analyze the components and working of Data Pre-processing for Natural Language Processing Tasks.\n5. Example: Evaluate the strengths and limitations of Data Pre-processing for Natural Language Processing Tasks.\n6. Example: Design an experiment or application using Data Pre-processing for Natural Language Processing Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Data Pre-processing for Natural Language Processing Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Bag of Words Approach and Word Embedding?\n2. Example: Can you explain the key ideas behind Bag of Words Approach and Word Embedding?\n3. Example: How would you apply Bag of Words Approach and Word Embedding in a real-world scenario?\n4. Example: Analyze the components and working of Bag of Words Approach and Word Embedding.\n5. Example: Evaluate the strengths and limitations of Bag of Words Approach and Word Embedding.\n6. Example: Design an experiment or application using Bag of Words Approach and Word Embedding.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Bag of Words Approach and Word Embedding. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Attention Mechanism in Transformers?\n2. Example: Can you explain the key ideas behind Attention Mechanism in Transformers?\n3. Example: How would you apply Attention Mechanism in Transformers in a real-world scenario?\n4. Example: Analyze the components and working of Attention Mechanism in Transformers.\n5. Example: Evaluate the strengths and limitations of Attention Mechanism in Transformers.\n6. Example: Design an experiment or application using Attention Mechanism in Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Attention Mechanism in Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Neural Machine Translation using Transformers?\n2. Example: Can you explain the key ideas behind Neural Machine Translation using Transformers?\n3. Example: How would you apply Neural Machine Translation using Transformers in a real-world scenario?\n4. Example: Analyze the components and working of Neural Machine Translation using Transformers.\n5. Example: Evaluate the strengths and limitations of Neural Machine Translation using Transformers.\n6. Example: Design an experiment or application using Neural Machine Translation using Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Neural Machine Translation using Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Encoder, Decoder, and Sequence-to-Sequence Transformers?\n2. Example: Can you explain the key ideas behind Encoder, Decoder, and Sequence-to-Sequence Transformers?\n3. Example: How would you apply Encoder, Decoder, and Sequence-to-Sequence Transformers in a real-world scenario?\n4. Example: Analyze the components and working of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n5. Example: Evaluate the strengths and limitations of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n6. Example: Design an experiment or application using Encoder, Decoder, and Sequence-to-Sequence Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Encoder, Decoder, and Sequence-to-Sequence Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n2. Example: Can you explain the key ideas behind Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n3. Example: How would you apply Pretraining, Finetuning, and Reinforcement Learning with Human Feedback in a real-world scenario?\n4. Example: Analyze the components and working of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n5. Example: Evaluate the strengths and limitations of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n6. Example: Design an experiment or application using Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Pretraining, Finetuning, and Reinforcement Learning with Human Feedback. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Prompt Engineering and Chain of Thought Prompting?\n2. Example: Can you explain the key ideas behind Prompt Engineering and Chain of Thought Prompting?\n3. Example: How would you apply Prompt Engineering and Chain of Thought Prompting in a real-world scenario?\n4. Example: Analyze the components and working of Prompt Engineering and Chain of Thought Prompting.\n5. Example: Evaluate the strengths and limitations of Prompt Engineering and Chain of Thought Prompting.\n6. Example: Design an experiment or application using Prompt Engineering and Chain of Thought Prompting.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Prompt Engineering and Chain of Thought Prompting. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are examples of questions corresponding to different Bloom’s levels:\n\n1. Example: What is Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n2. Example: Can you explain the key ideas behind Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n3. Example: How would you apply Natural Language Processing Tasks and Transformer Architectures Used for the Tasks in a real-world scenario?\n4. Example: Analyze the components and working of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n5. Example: Evaluate the strengths and limitations of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n6. Example: Design an experiment or application using Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Natural Language Processing Tasks and Transformer Architectures Used for the Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.'],
    'PS4': ['You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Decision Tree Models?\n2. Skill: Understand, Example: Can you explain the key ideas behind Decision Tree Models?\n3. Skill: Apply, Example: How would you apply Decision Tree Models in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Decision Tree Models.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Decision Tree Models.\n6. Skill: Create, Example: Design an experiment or application using Decision Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Decision Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Training, Validation, and Testing of Machine Learning Models?\n2. Skill: Understand, Example: Can you explain the key ideas behind Training, Validation, and Testing of Machine Learning Models?\n3. Skill: Apply, Example: How would you apply Training, Validation, and Testing of Machine Learning Models in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Training, Validation, and Testing of Machine Learning Models.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Training, Validation, and Testing of Machine Learning Models.\n6. Skill: Create, Example: Design an experiment or application using Training, Validation, and Testing of Machine Learning Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Training, Validation, and Testing of Machine Learning Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Gradient Boosted Tree Models?\n2. Skill: Understand, Example: Can you explain the key ideas behind Gradient Boosted Tree Models?\n3. Skill: Apply, Example: How would you apply Gradient Boosted Tree Models in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Gradient Boosted Tree Models.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Gradient Boosted Tree Models.\n6. Skill: Create, Example: Design an experiment or application using Gradient Boosted Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Gradient Boosted Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Linear Regression, Logistic Regression, and Multilayer Perceptron?\n2. Skill: Understand, Example: Can you explain the key ideas behind Linear Regression, Logistic Regression, and Multilayer Perceptron?\n3. Skill: Apply, Example: How would you apply Linear Regression, Logistic Regression, and Multilayer Perceptron in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n6. Skill: Create, Example: Design an experiment or application using Linear Regression, Logistic Regression, and Multilayer Perceptron.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Linear Regression, Logistic Regression, and Multilayer Perceptron. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Stochastic Gradient Descent?\n2. Skill: Understand, Example: Can you explain the key ideas behind Stochastic Gradient Descent?\n3. Skill: Apply, Example: How would you apply Stochastic Gradient Descent in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Stochastic Gradient Descent.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Stochastic Gradient Descent.\n6. Skill: Create, Example: Design an experiment or application using Stochastic Gradient Descent.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Stochastic Gradient Descent. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Backpropagation?\n2. Skill: Understand, Example: Can you explain the key ideas behind Backpropagation?\n3. Skill: Apply, Example: How would you apply Backpropagation in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Backpropagation.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Backpropagation.\n6. Skill: Create, Example: Design an experiment or application using Backpropagation.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Backpropagation. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Foundations of Computer Vision and Convolutional Neural Networks?\n2. Skill: Understand, Example: Can you explain the key ideas behind Foundations of Computer Vision and Convolutional Neural Networks?\n3. Skill: Apply, Example: How would you apply Foundations of Computer Vision and Convolutional Neural Networks in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Foundations of Computer Vision and Convolutional Neural Networks.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Foundations of Computer Vision and Convolutional Neural Networks.\n6. Skill: Create, Example: Design an experiment or application using Foundations of Computer Vision and Convolutional Neural Networks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Foundations of Computer Vision and Convolutional Neural Networks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Transfer Learning for Computer Vision?\n2. Skill: Understand, Example: Can you explain the key ideas behind Transfer Learning for Computer Vision?\n3. Skill: Apply, Example: How would you apply Transfer Learning for Computer Vision in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Transfer Learning for Computer Vision.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Transfer Learning for Computer Vision.\n6. Skill: Create, Example: Design an experiment or application using Transfer Learning for Computer Vision.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Transfer Learning for Computer Vision. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Image Segmentation and Object Detection?\n2. Skill: Understand, Example: Can you explain the key ideas behind Image Segmentation and Object Detection?\n3. Skill: Apply, Example: How would you apply Image Segmentation and Object Detection in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Image Segmentation and Object Detection.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Image Segmentation and Object Detection.\n6. Skill: Create, Example: Design an experiment or application using Image Segmentation and Object Detection.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Image Segmentation and Object Detection. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Data Pre-processing for Natural Language Processing Tasks?\n2. Skill: Understand, Example: Can you explain the key ideas behind Data Pre-processing for Natural Language Processing Tasks?\n3. Skill: Apply, Example: How would you apply Data Pre-processing for Natural Language Processing Tasks in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Data Pre-processing for Natural Language Processing Tasks.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Data Pre-processing for Natural Language Processing Tasks.\n6. Skill: Create, Example: Design an experiment or application using Data Pre-processing for Natural Language Processing Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Data Pre-processing for Natural Language Processing Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Bag of Words Approach and Word Embedding?\n2. Skill: Understand, Example: Can you explain the key ideas behind Bag of Words Approach and Word Embedding?\n3. Skill: Apply, Example: How would you apply Bag of Words Approach and Word Embedding in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Bag of Words Approach and Word Embedding.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Bag of Words Approach and Word Embedding.\n6. Skill: Create, Example: Design an experiment or application using Bag of Words Approach and Word Embedding.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Bag of Words Approach and Word Embedding. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Attention Mechanism in Transformers?\n2. Skill: Understand, Example: Can you explain the key ideas behind Attention Mechanism in Transformers?\n3. Skill: Apply, Example: How would you apply Attention Mechanism in Transformers in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Attention Mechanism in Transformers.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Attention Mechanism in Transformers.\n6. Skill: Create, Example: Design an experiment or application using Attention Mechanism in Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Attention Mechanism in Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Neural Machine Translation using Transformers?\n2. Skill: Understand, Example: Can you explain the key ideas behind Neural Machine Translation using Transformers?\n3. Skill: Apply, Example: How would you apply Neural Machine Translation using Transformers in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Neural Machine Translation using Transformers.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Neural Machine Translation using Transformers.\n6. Skill: Create, Example: Design an experiment or application using Neural Machine Translation using Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Neural Machine Translation using Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Encoder, Decoder, and Sequence-to-Sequence Transformers?\n2. Skill: Understand, Example: Can you explain the key ideas behind Encoder, Decoder, and Sequence-to-Sequence Transformers?\n3. Skill: Apply, Example: How would you apply Encoder, Decoder, and Sequence-to-Sequence Transformers in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n6. Skill: Create, Example: Design an experiment or application using Encoder, Decoder, and Sequence-to-Sequence Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Encoder, Decoder, and Sequence-to-Sequence Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n2. Skill: Understand, Example: Can you explain the key ideas behind Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n3. Skill: Apply, Example: How would you apply Pretraining, Finetuning, and Reinforcement Learning with Human Feedback in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n6. Skill: Create, Example: Design an experiment or application using Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Pretraining, Finetuning, and Reinforcement Learning with Human Feedback. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Prompt Engineering and Chain of Thought Prompting?\n2. Skill: Understand, Example: Can you explain the key ideas behind Prompt Engineering and Chain of Thought Prompting?\n3. Skill: Apply, Example: How would you apply Prompt Engineering and Chain of Thought Prompting in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Prompt Engineering and Chain of Thought Prompting.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Prompt Engineering and Chain of Thought Prompting.\n6. Skill: Create, Example: Design an experiment or application using Prompt Engineering and Chain of Thought Prompting.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Prompt Engineering and Chain of Thought Prompting. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are Bloom’s skills and example questions corresponding to different levels:\n\n1. Skill: Remember, Example: What is Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n2. Skill: Understand, Example: Can you explain the key ideas behind Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n3. Skill: Apply, Example: How would you apply Natural Language Processing Tasks and Transformer Architectures Used for the Tasks in a real-world scenario?\n4. Skill: Analyze, Example: Analyze the components and working of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n5. Skill: Evaluate, Example: Evaluate the strengths and limitations of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n6. Skill: Create, Example: Design an experiment or application using Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Natural Language Processing Tasks and Transformer Architectures Used for the Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.'],
    'PS5': ['You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Decision Tree Models?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Decision Tree Models?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Decision Tree Models in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Decision Tree Models.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Decision Tree Models.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Decision Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Decision Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Training, Validation, and Testing of Machine Learning Models?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Training, Validation, and Testing of Machine Learning Models?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Training, Validation, and Testing of Machine Learning Models in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Training, Validation, and Testing of Machine Learning Models.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Training, Validation, and Testing of Machine Learning Models.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Training, Validation, and Testing of Machine Learning Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Training, Validation, and Testing of Machine Learning Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Gradient Boosted Tree Models?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Gradient Boosted Tree Models?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Gradient Boosted Tree Models in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Gradient Boosted Tree Models.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Gradient Boosted Tree Models.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Gradient Boosted Tree Models.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Gradient Boosted Tree Models. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Linear Regression, Logistic Regression, and Multilayer Perceptron?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Linear Regression, Logistic Regression, and Multilayer Perceptron?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Linear Regression, Logistic Regression, and Multilayer Perceptron in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Linear Regression, Logistic Regression, and Multilayer Perceptron.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Linear Regression, Logistic Regression, and Multilayer Perceptron.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Linear Regression, Logistic Regression, and Multilayer Perceptron. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Stochastic Gradient Descent?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Stochastic Gradient Descent?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Stochastic Gradient Descent in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Stochastic Gradient Descent.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Stochastic Gradient Descent.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Stochastic Gradient Descent.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Stochastic Gradient Descent. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Backpropagation?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Backpropagation?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Backpropagation in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Backpropagation.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Backpropagation.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Backpropagation.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Backpropagation. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Foundations of Computer Vision and Convolutional Neural Networks?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Foundations of Computer Vision and Convolutional Neural Networks?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Foundations of Computer Vision and Convolutional Neural Networks in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Foundations of Computer Vision and Convolutional Neural Networks.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Foundations of Computer Vision and Convolutional Neural Networks.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Foundations of Computer Vision and Convolutional Neural Networks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Foundations of Computer Vision and Convolutional Neural Networks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Transfer Learning for Computer Vision?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Transfer Learning for Computer Vision?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Transfer Learning for Computer Vision in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Transfer Learning for Computer Vision.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Transfer Learning for Computer Vision.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Transfer Learning for Computer Vision.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Transfer Learning for Computer Vision. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Image Segmentation and Object Detection?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Image Segmentation and Object Detection?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Image Segmentation and Object Detection in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Image Segmentation and Object Detection.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Image Segmentation and Object Detection.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Image Segmentation and Object Detection.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Image Segmentation and Object Detection. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Data Pre-processing for Natural Language Processing Tasks?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Data Pre-processing for Natural Language Processing Tasks?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Data Pre-processing for Natural Language Processing Tasks in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Data Pre-processing for Natural Language Processing Tasks.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Data Pre-processing for Natural Language Processing Tasks.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Data Pre-processing for Natural Language Processing Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Data Pre-processing for Natural Language Processing Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Bag of Words Approach and Word Embedding?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Bag of Words Approach and Word Embedding?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Bag of Words Approach and Word Embedding in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Bag of Words Approach and Word Embedding.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Bag of Words Approach and Word Embedding.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Bag of Words Approach and Word Embedding.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Bag of Words Approach and Word Embedding. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Attention Mechanism in Transformers?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Attention Mechanism in Transformers?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Attention Mechanism in Transformers in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Attention Mechanism in Transformers.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Attention Mechanism in Transformers.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Attention Mechanism in Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Attention Mechanism in Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Neural Machine Translation using Transformers?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Neural Machine Translation using Transformers?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Neural Machine Translation using Transformers in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Neural Machine Translation using Transformers.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Neural Machine Translation using Transformers.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Neural Machine Translation using Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Neural Machine Translation using Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Encoder, Decoder, and Sequence-to-Sequence Transformers?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Encoder, Decoder, and Sequence-to-Sequence Transformers?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Encoder, Decoder, and Sequence-to-Sequence Transformers in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Encoder, Decoder, and Sequence-to-Sequence Transformers.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Encoder, Decoder, and Sequence-to-Sequence Transformers.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Encoder, Decoder, and Sequence-to-Sequence Transformers. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Pretraining, Finetuning, and Reinforcement Learning with Human Feedback?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Pretraining, Finetuning, and Reinforcement Learning with Human Feedback in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Pretraining, Finetuning, and Reinforcement Learning with Human Feedback.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Pretraining, Finetuning, and Reinforcement Learning with Human Feedback. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Prompt Engineering and Chain of Thought Prompting?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Prompt Engineering and Chain of Thought Prompting?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Prompt Engineering and Chain of Thought Prompting in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Prompt Engineering and Chain of Thought Prompting.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Prompt Engineering and Chain of Thought Prompting.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Prompt Engineering and Chain of Thought Prompting.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Prompt Engineering and Chain of Thought Prompting. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.', 'You are an experienced data science instructor for a graduate-level course in an engineering program.\n\nThe following are the revised Bloom’s skill levels, their explanation and questions corresponding to each level:\n\n1. Skill: Remember, Explanation: Retrieve relevant knowledge from long-term memory, Example: What is Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n2. Skill: Understand, Explanation: Construct meaning from instructional messages, including oral, written and graphic communication, Example: Can you explain the key ideas behind Natural Language Processing Tasks and Transformer Architectures Used for the Tasks?\n3. Skill: Apply, Explanation: Carry out or use a procedure in a given situation, Example: How would you apply Natural Language Processing Tasks and Transformer Architectures Used for the Tasks in a real-world scenario?\n4. Skill: Analyze, Explanation: Break material into foundational parts and determine how parts relate to one another and the overall structure or purpose, Example: Analyze the components and working of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n5. Skill:Evaluate, Explanation: Make judgments based on criteria and standards, Example: Evaluate the strengths and limitations of Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n6. Skill: Create, Explanation: Put elements together to form a coherent whole; reorganize into a new pattern or structure, Example: Design an experiment or application using Natural Language Processing Tasks and Transformer Architectures Used for the Tasks.\n\nYou are supposed to create one question corresponding to each level in revised Bloom’s taxonomy for the Natural Language Processing Tasks and Transformer Architectures Used for the Tasks. \nThese questions are created to evaluate students on a range of cognitive skills, from basic knowledge to critical thinking and problem-solving.\n\nPlease make sure that the questions are relatable to students in India.'],
}

# Load the workbook
wb = load_workbook(file_path)

for sheet_name, values in data_to_fill.items():
    if sheet_name not in wb.sheetnames:
        print(f"Sheet '{sheet_name}' not found. Skipping.")
        continue

    ws = wb[sheet_name]
    merged_ranges = ws.merged_cells.ranges

    filled = 0
    row = 2  # Start from row 2 to skip header

    while filled < len(values):
        cell = ws.cell(row=row, column=2)  # Column B

        # Check if this cell is the top-left of a merged range
        is_top_left = True
        for mrange in merged_ranges:
            if cell.coordinate in mrange:
                if cell.coordinate != mrange.start_cell.coordinate:
                    is_top_left = False
                break

        if is_top_left:
            cell.value = values[filled]
            filled += 1

        row += 1

# Save the workbook
wb.save(file_path)
print("Data filled successfully, starting from row 2.")


Data filled successfully, starting from row 2.
